In [5]:
import ee
import eemont
import numpy as np
import pandas as pd
import swifter

from datetime import datetime
from ee import EEException
from tqdm import tqdm

In [2]:
# authenticating ee with google account
ee.Authenticate(quiet=True)
# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

Read in prepared allen coral atlas file.

In [3]:
allen_sample = pd.read_pickle('..\data\mesoamerica_subset.pkl')
allen_sample
allen_sample.to_csv("allen_sample.csv", index=False)

# Collect Satellite Data

Queries Google Earth Engine to get data from either Landsat8 or MODIS. Multiple satellite data points can be returned from a single query, so the data point that occured the earliest is used. Arguments that need to be specified are satellite collection, start date, end date, spectral bands, and scale. Only Landsat can calculate spectral indicies, so that argument is None for modis.

In [19]:
# Function to return an empty row for given location
def noBandsFound(collection, bands, spec_idxs, lat, lon):
	new_row = pd.DataFrame()
	new_row = new_row.reindex(columns=['lat', 'long', *bands], fill_value=np.nan)
	if(collection == 'LANDSAT/LC08/C02/T1_L2'):
		new_row = new_row.reindex(columns=[*new_row.columns.tolist(), *spec_idxs], fill_value=np.nan)
	new_row['datetime_' + collection] = np.nan
	new_row['lat'] = lat
	new_row['long'] = lon
	return new_row.T.squeeze()

In [20]:
def getSatelliteData(row, collection, start_date, end_date, bands, spec_idxs, scale):
    row = np.squeeze(row)
    try:
        if (collection != 'LANDSAT/LC08/C02/T1_L2'):
            img_col = ee.ImageCollection(collection)\
                        .select(bands)
        else:
            img_col = ee.ImageCollection(collection)\
                        .select(bands)\
                        .spectralIndices(spec_idxs)

        data = img_col\
                 .filterBounds(geometry=ee.Geometry.Point(row['long'], row['lat']))\
                 .filterDate(start_date, end_date)\
                 .getRegion(geometry=ee.Geometry.Point(row['long'], row['lat']), scale=scale)\
                 .getInfo()

        df = pd.DataFrame(data[1:], columns=data[0])
        df.dropna(inplace=True)
        df = df.dropna()  
        df['datetime_' + collection] = df.time.apply(lambda x: datetime.utcfromtimestamp(x / 1000))
        df['long'] = row['long']
        df['lat'] = row['lat']
        row = row.to_frame().T

        if df.shape[0] > 1:
                # Loop through data return from Google Earth Engine query and find data point closest to the coral date. 
                # Select only the earliest Satellite data point
                time_deltas = {}
                for i in range(df.shape[0]):
                    time_deltas[i] = np.abs(pd.to_datetime(df['datetime_' + collection].values[i]) - datetime.strptime(start_date, '%Y-%m-%d'))
                ind = min(time_deltas, key=time_deltas.get)

                data = df.iloc[ind,].to_frame().T
                return data.drop(columns=['id', 'time', 'longitude', 'latitude']).squeeze()

		 # We only have one Landsat data point for the selected time interval and region
        elif df.shape[0]==1:
            data = df.iloc[0,].to_frame().T
            return data.drop(columns=['id', 'time', 'longitude', 'latitude']).squeeze()

		# No data was found for given location and date range
        elif df.shape[0]==0:
            return noBandsFound(collection, bands, spec_idxs, row['lat'], row['lon'])
			
    # Error occured when retrieving data
    except EEException as e:
        return noBandsFound(collection, bands, spec_idxs, row['lat'], row['lon'])
        

## MODIS
Collected in 2 day intervals since MODIS covered the earth every 1 to 2 days.

In [ ]:
# Initialize Arguments
start_date = '2021-01-01'
end_date = ee.Date(start_date).advance(2, 'day')
bands = ['sur_refl_b08', 'sur_refl_b09', 'sur_refl_b10', 'sur_refl_b11', 
         'sur_refl_b12', 'sur_refl_b13', 'sur_refl_b14', 'sur_refl_b15', 'sur_refl_b16']
scale = 1000

# Get the Modis data
modis_df = allen_sample.swifter.apply(getSatelliteData, args = ('MODIS/006/MYDOCGA', start_date, end_date, bands, None, scale), axis = 1)

In [ ]:
modis_df

In [8]:
modis_df.to_pickle('./files/modis_caribbean.pkl')

## LANDSAT
Needs to be collected monthly due to 16-day orbit cycles.

In [14]:
# Initialize Arguments
start_date = '2022-03-01'
end_date = ee.Date(start_date).advance(1, 'month')
spec_idxs = ['AWEInsh', 'AWEIsh', 'LSWI', 'MBWI', 'MLSWI26', 'MLSWI27',
            'MNDWI', 'MuWIR', 'NDVIMNDWI', 'NDWI', 'NDWIns', 'NWI', 'SWM', 'WI1', 'WI2', 'WRI']
bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
scale = 100

# Get the Landsat data
landsat_df = allen_sample.swifter.apply(getSatelliteData, args = ('LANDSAT/LC08/C02/T1_L2', start_date, end_date, bands, spec_idxs, scale), axis = 1)


AttributeError: type object 'datetime.datetime' has no attribute 'timezone'

In [6]:
# Read the CSV file
modis_csv = pd.read_csv("..\data\MODIS_Sampled.csv")
landsat_csv = pd.read_csv("..\data\LANDSAT_Sampled.csv")
turbid_csv = pd.read_csv("..\data\MODIS_Kd490_Estimated.csv")
# convert to pickle
modis_csv.to_pickle("modis_sampled.pkl")
landsat_csv.to_pickle("landsat_sampled.pkl")
turbid_csv.to_pickle('turbidity_sampled.pkl')
# read in Pickle files
modis_df = pd.read_pickle("modis_sampled.pkl")
landsat_df = pd.read_pickle('landsat_sampled.pkl')
turbid_df = pd.read_pickle('turbidity_sampled.pkl')


In [29]:
modis_df

,system:index,centroid_column,class,lat,long,sur_refl_b08,sur_refl_b09,sur_refl_b10,sur_refl_b11,sur_refl_b12,sur_refl_b13,sur_refl_b14,sur_refl_b15,sur_refl_b16,.geo
0,00000000000000001ac3_0,"{""type"":""Point"",""coordinates"":[-88.12947315863...",Coral/Algae,16.833359,-88.129473,526.5,542.0,627.5,624.5,622.5,378.5,381.5,369.5,369.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
1,00000000000000001acb_0,"{""type"":""Point"",""coordinates"":[-86.29617278211...",Coral/Algae,16.400705,-86.296173,1090.5,1097.5,1141.0,-24.0,-29.0,-61.5,-58.5,-56.0,-45.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2,00000000000000001bc1_0,"{""type"":""Point"",""coordinates"":[-85.86157550918...",Coral/Algae,16.444167,-85.861576,321.5,346.0,431.0,437.0,438.0,204.5,211.0,329.5,-100.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
3,00000000000000001c2b_0,"{""type"":""Point"",""coordinates"":[-87.99509831018...",Coral/Algae,17.818673,-87.995098,358.0,441.0,602.0,587.5,579.0,73.5,70.0,-7.5,9.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
4,00000000000000001c3f_0,"{""type"":""Point"",""coordinates"":[-87.59541364023...",Coral/Algae,17.225310,-87.595414,584.0,570.0,602.5,543.5,529.0,272.0,268.5,237.0,229.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,0000000000000000bef7_0,"{""type"":""Point"",""coordinates"":[-87.76983233650...",Non-Coral,17.616940,-87.769832,872.5,942.5,1096.0,1100.0,1109.5,480.5,459.5,265.5,239.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
350,0000000000000000bfe6_0,"{""type"":""Point"",""coordinates"":[-88.18063481308...",Non-Coral,16.265765,-88.180635,1449.5,1510.0,110.0,101.5,95.0,-35.5,-36.5,-47.5,-49.5,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
351,0000000000000000bff8_0,"{""type"":""Point"",""coordinates"":[-87.45508031583...",Non-Coral,19.389883,-87.455080,2920.0,2977.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
352,0000000000000000c1e8_0,"{""type"":""Point"",""coordinates"":[-80.33224779975...",Non-Coral,14.305191,-80.332248,1691.0,1798.0,1995.0,1785.0,-100.0,-100.0,-100.0,784.0,-100.0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."


In [28]:
landsat_df

,system:index,NDVI,NDWI,QA_PIXEL,Blue,Green,Red,Near Infrared,Shortwave Infrared 1,Shortwave Infrared 2,centroid_column,class,lat,long,.geo
0,00000000000000001aa0_0,0.010076,0.025037,22116.0,18080.5,17891.5,16678.0,17017.5,16467.5,14973.0,"{""type"":""Point"",""coordinates"":[-88.07478980683...",Coral/Algae,17.285198,-88.074790,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
1,00000000000000001abb_0,0.061934,0.056526,21985.0,7299.0,8803.5,6944.5,7861.5,9638.5,9665.0,"{""type"":""Point"",""coordinates"":[-82.37834119129...",Coral/Algae,15.053791,-82.378341,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2,00000000000000001abb_1,0.085535,0.034736,21952.0,5487.5,7618.5,5987.0,7107.0,8883.5,8820.0,"{""type"":""Point"",""coordinates"":[-82.37834119129...",Coral/Algae,15.053791,-82.378341,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
3,00000000000000001abb_2,0.061654,0.079080,21952.0,6590.5,8504.5,6415.0,7258.0,8935.0,8874.5,"{""type"":""Point"",""coordinates"":[-82.37834119129...",Coral/Algae,15.053791,-82.378341,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
4,00000000000000001abb_3,0.018967,0.132752,21952.0,8144.5,9736.0,7176.5,7454.0,8903.0,8829.5,"{""type"":""Point"",""coordinates"":[-82.37834119129...",Coral/Algae,15.053791,-82.378341,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36777,0000000000000000c34d_7,0.038699,0.008676,22116.0,11881.5,14010.0,12743.0,13769.0,13617.5,12740.0,"{""type"":""Point"",""coordinates"":[-87.82314239205...",Non-Coral,18.281875,-87.823142,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
36778,0000000000000000c34d_8,0.078100,-0.036359,22116.0,9735.5,12052.5,11084.0,12962.0,13510.0,12696.5,"{""type"":""Point"",""coordinates"":[-87.82314239205...",Non-Coral,18.281875,-87.823142,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
36779,0000000000000000c34d_9,0.078644,-0.034282,22116.0,19153.0,11887.5,10875.0,12731.5,13422.5,12537.5,"{""type"":""Point"",""coordinates"":[-87.82314239205...",Non-Coral,18.281875,-87.823142,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
36780,0000000000000000c34d_10,0.061643,-0.013270,22116.0,10713.5,13199.0,11980.0,13554.0,13550.0,12757.0,"{""type"":""Point"",""coordinates"":[-87.82314239205...",Non-Coral,18.281875,-87.823142,"{""geodesic"":false,""type"":""Point"",""coordinates""..."


In [26]:
# Column renaming
landsat_df.rename({'SR_B2' : 'Blue',
                        'SR_B3' : 'Green',
                        'SR_B4' : 'Red',
                        'SR_B5' : 'Near Infrared',
                        'SR_B6' : 'Shortwave Infrared 1',
                        'SR_B7' : 'Shortwave Infrared 2'}, axis = 1, inplace = True)

In [27]:
turbid_df

,system:index,Kd490,centroid_column,class,lat,long,.geo
0,00000000000000001acb_0,0.007302,"{""type"":""Point"",""coordinates"":[-86.29617278211...",Coral/Algae,16.400705,-86.296173,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
1,00000000000000003e3e_0,0.017908,"{""type"":""Point"",""coordinates"":[-83.45743659712...",Coral/Algae,12.487995,-83.457437,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2,00000000000000006515_0,0.004498,"{""type"":""Point"",""coordinates"":[-87.80118289574...",Coral/Algae,17.298629,-87.801183,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
3,000000000000000071ca_0,0.004745,"{""type"":""Point"",""coordinates"":[-85.82401733027...",Coral/Algae,16.511279,-85.824017,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
4,00000000000000007762_0,0.008234,"{""type"":""Point"",""coordinates"":[-86.18631178742...",Coral/Algae,16.440332,-86.186312,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
5,000000000000000079de_0,0.005413,"{""type"":""Point"",""coordinates"":[-85.82587704011...",Coral/Algae,16.474190,-85.825877,"{""geodesic"":false,""type"":""Point"",""coordinates""..."


In [15]:
landsat_df.to_pickle('../data/landsat_mesoamerica.pkl')

## VIIRS
Note: Was not used in this project. Could be used in the future

In [14]:
# Initialize Arguments
start_date = '2022-03-01'
end_date = ee.Date(start_date).advance(1, 'day')
bands = ['M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'I1', 'I2']
scale = 1000

# Get the VIIRS data
viirs_df = allen_sample.swifter.apply(getSatelliteData, args = ('NOAA/VIIRS/001/VNP09GA', start_date, end_date, bands, None, scale), axis = 1)

Pandas Apply:   0%|          | 0/1550 [00:00<?, ?it/s]

In [15]:
viirs_df

M1    M2    M3    M4    M5    M7    I1    I2  \
class                                                                 
Coral/Algae 2971790  1785  1804  1851  1986  1777  2457  3061  3588   
            2118004  2559  2575  2670  2737  2397  2463  1375  1422   
            2118268  1976  1960  1955  2084  1840  2354  4893  5260   
            2097168  3092  3124  3258  3305  3113  3293  2756  2959   
            2111207  3092  3124  3258  3305  3113  3293  2723  2857   
...                   ...   ...   ...   ...   ...   ...   ...   ...   
Non-Coral   2100310  2131  2166  2335  2305  2161  2153  3979  4234   
            2971962  7016  7301  7590  7909  8218  8565  8342  8826   
            2108374  5343  5466  5635  5801  5824  6216  5315  5561   
            2115402  2131  2166  2335  2305  2161  2153  3979  4234   
            2100315  2559  2575  2670  2737  2397  2463  1375  1422   

                    datetime_NOAA/VIIRS/001/VNP09GA       long        lat  
class                                                                      
Coral/Algae 2971790                      2022-03-01 -78.609698  26.523708  
            2118004                      2022-03-01 -78.594745  26.514949  
            2118268                      2022-03-01 -78.587069  26.529069  
            2097168                      2022-03-01 -78.612738  26.514202  
            2111207                      2022-03-01 -78.611099  26.514786  
...                                             ...        ...        ...  
Non-Coral   2100310                      2022-03-01 -78.590252  26.515946  
            2971962                      2022-03-01 -78.584200  26.520686  
            2108374                      2022-03-01 -78.580451  26.527640  
            2115402                      2022-03-01 -78.593178  26.514068  
            2100315                      2022-03-01 -78.594690  26.513633  

[1550 rows x 11 columns]

In [16]:
viirs_df.to_pickle('./files/viirs_caribbean.pkl')

## Merging
Join satellite data on longitude and latitude and then join with coral data.

In [34]:
# print(set(allen_sample.columns), set(all_satellite.columns))

{'long', 'class', 'geometry', 'centroid_column', 'lat'} {'NDWI', '.geo', 'sur_refl_b11', 'NDVI', 'sur_refl_b12', 'sur_refl_b16', 'Shortwave Infrared 2', 'Green', 'sur_refl_b08', 'Blue', 'sur_refl_b09', 'sur_refl_b15', 'long', 'system:index', 'Near Infrared', 'Kd490', 'centroid_column', 'sur_refl_b14', 'sur_refl_b13', 'sur_refl_b10', 'class', 'Shortwave Infrared 1', 'QA_PIXEL', 'Red', 'lat'}


In [45]:
from functools import reduce

allen_sample = pd.read_pickle('..\data\mesoamerica_subset.pkl')
#modis_df = pd.read_pickle("modis_sampled.pkl")
#landsat_df = pd.read_pickle('landsat_sampled.pkl')
#turbid_df = pd.read_pickle('turbidity_sampled.pkl')

# Define the columns that may cause merge issues
conflicting_cols = ['centroid_column', 'system:index', '.geo']

# Clean each satellite DataFrame
def clean_satellite_df(df):
    return df.drop(columns=[col for col in conflicting_cols if col in df.columns], errors='ignore')

# Apply cleaning before merge
modis_df_clean = clean_satellite_df(modis_df)
landsat_df_clean = clean_satellite_df(landsat_df)
turbid_df_clean = clean_satellite_df(turbid_df)
def safe_reset_index(df):
    if 'class' in df.index.names or df.index.nlevels > 1:
        return df.reset_index()
    return df.reset_index(drop=True)

modis_df_clean = safe_reset_index(modis_df_clean)
landsat_df_clean = safe_reset_index(landsat_df_clean)
turbid_df_clean = safe_reset_index(turbid_df_clean)

print(modis_df_clean['class'].unique())
print(landsat_df_clean['class'].unique())


all_satellite = reduce(
    lambda left, right: pd.merge(left, right, on=['lat', 'long', 'class'], how='inner'),
    [modis_df_clean, landsat_df_clean]
)
print(all_satellite['class'].unique())


# Step 2: Merge with allen_sample
allen_sample = allen_sample.reset_index(drop=True)
data_merged = allen_sample.merge(all_satellite, on=['lat', 'long', 'class'], how='inner')


# Step 3: Keep only desired columns
desired_columns = [
    'class', 'geometry', 'centroid_column', 'long', 'lat',
    'sur_refl_b08', 'sur_refl_b09', 'sur_refl_b10', 'sur_refl_b11', 'sur_refl_b12',
    'sur_refl_b13', 'sur_refl_b14', 'sur_refl_b15', 'sur_refl_b16',
    'datetime_MODIS/006/MYDOCGA', 'Blue', 'Green', 'Red', 'Near Infrared',
    'Shortwave Infrared 1', 'Shortwave Infrared 2', 'QA_PIXEL', 'AWEInsh',
    'AWEIsh', 'LSWI', 'MBWI', 'MLSWI26', 'MLSWI27', 'MNDWI', 'MuWIR',
    'NDVIMNDWI', 'NDWI', 'NDWIns', 'NWI', 'SWM', 'WI1', 'WI2', 'WRI',
    'datetime_LANDSAT/LC08/C02/T1_L2'
]

# Only retain desired columns
final_data = data_merged.loc[:, data_merged.columns.intersection(desired_columns)]
print(final_data['class'].unique())

['Coral/Algae' 'Non-Coral']
['Coral/Algae' 'Non-Coral']
['Coral/Algae' 'Non-Coral']
['Coral/Algae' 'Non-Coral']


In [46]:
final_data

,class,geometry,centroid_column,long,lat,sur_refl_b08,sur_refl_b09,sur_refl_b10,sur_refl_b11,sur_refl_b12,...,sur_refl_b15,sur_refl_b16,NDWI,QA_PIXEL,Blue,Green,Red,Near Infrared,Shortwave Infrared 1,Shortwave Infrared 2
0,Coral/Algae,"POLYGON ((-88.12662 16.83317, -88.12662 16.833...",POINT (-88.12947 16.83336),-88.129473,16.833359,526.5,542.0,627.5,624.5,622.5,...,369.5,369.0,0.021106,22180.0,28448.0,27830.5,27003.0,26680.0,20399.5,17362.5
1,Coral/Algae,"POLYGON ((-88.12662 16.83317, -88.12662 16.833...",POINT (-88.12947 16.83336),-88.129473,16.833359,526.5,542.0,627.5,624.5,622.5,...,369.5,369.0,0.021252,22116.0,27672.5,27270.5,26400.5,26135.5,20462.0,17811.0
2,Coral/Algae,"POLYGON ((-88.12662 16.83317, -88.12662 16.833...",POINT (-88.12947 16.83336),-88.129473,16.833359,526.5,542.0,627.5,624.5,622.5,...,369.5,369.0,0.021759,22116.0,28211.0,27752.5,26887.0,26570.5,20302.5,17098.0
3,Coral/Algae,"POLYGON ((-88.12662 16.83317, -88.12662 16.833...",POINT (-88.12947 16.83336),-88.129473,16.833359,526.5,542.0,627.5,624.5,622.5,...,369.5,369.0,0.022495,22116.0,26920.5,26340.5,25550.0,25181.5,19467.5,17153.5
4,Coral/Algae,"POLYGON ((-88.12662 16.83317, -88.12662 16.833...",POINT (-88.12947 16.83336),-88.129473,16.833359,526.5,542.0,627.5,624.5,622.5,...,369.5,369.0,0.023003,22116.0,27001.5,26394.5,25631.0,25207.5,19290.0,16939.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7017,Non-Coral,"POLYGON ((-88.09401 16.38868, -88.0941 16.3886...",POINT (-88.09398 16.39153),-88.093979,16.391533,201.0,220.0,296.0,271.0,259.5,...,-1.0,1.0,0.143401,21952.0,9306.0,9608.0,7511.0,7198.0,7415.0,7456.0
7018,Non-Coral,"POLYGON ((-88.09401 16.38868, -88.0941 16.3886...",POINT (-88.09398 16.39153),-88.093979,16.391533,201.0,220.0,296.0,271.0,259.5,...,-1.0,1.0,0.150077,21952.0,9550.0,9694.0,7426.0,7164.0,7426.0,7453.0
7019,Non-Coral,"POLYGON ((-88.09401 16.38868, -88.0941 16.3886...",POINT (-88.09398 16.39153),-88.093979,16.391533,201.0,220.0,296.0,271.0,259.5,...,-1.0,1.0,0.148093,21952.0,9182.0,10086.0,8693.0,7484.0,7604.0,7566.0
7020,Non-Coral,"POLYGON ((-88.09401 16.38868, -88.0941 16.3886...",POINT (-88.09398 16.39153),-88.093979,16.391533,201.0,220.0,296.0,271.0,259.5,...,-1.0,1.0,0.150116,21952.0,9448.0,9688.0,7473.0,7159.0,7428.0,7464.0


In [47]:
final_data.columns
final_data['class'].unique()

array(['Coral/Algae', 'Non-Coral'], dtype=object)

In [48]:
final_data.to_pickle('../data/merged_satellite_allen_mesoamerica.pkl')

# Bleaching Features
This section queries Google Earth Engine for chlor_a, nflh, poc, and sst data. The returned data is then used to build more features that are essential to our bleaching model.

In [48]:
# Importing the GCBD dataset that includes data on latitudes and longitudes for both the 
# Great Barrier Reef and the Caribbean 
df_GCBD = pd.read_csv('Global_Coral_Bleaching_Database.csv',index_col=0)
df_GCBD.rename({'Longitude_Degrees' : 'long',
                   'Latitude_Degrees' : 'lat',}, axis = 1, inplace = True)
df_GCBD

,COUNTRY,LOCATION,SITE_NAME,LATITUDE,LONGITUDE,DAY,MONTH,YEAR,DEPTH,PERCENT_BLEACHED,PERCENT_MORTALITY,SURVEY_TYPE,SOURCE,CITATION,COMMENTS,DATA_POINT_OF_CONTACT,POC_E-MAIL_ADDRESS,CORAL_REGIONS,MIN_PERCENT_BLEACHED,MAX_PERCENT_BLEACHED
RECORD_ID,,,,,,,,,,,,,,,,,,,,
100340.0,Belize,Ambergris Cay,NaN,18.033333,-87.883333,NaN,NaN,1995,NaN,NaN,NaN,NaN,ReefBase,NaN,Widespread but patchy,NaN,NaN,Caribbean/GoM,NaN,NaN
100341.0,Belize,NaN,NaN,17.316667,-88.000000,NaN,9.0,1995,42036,NaN,NaN,NaN,ReefBase,NaN,The episode this year appears to be the first ...,NaN,NaN,Caribbean/GoM,NaN,NaN
100342.0,Belize,Carrie Bow Cay,NaN,16.800000,-88.083333,NaN,NaN,1995,NaN,NaN,NaN,NaN,ReefBase,NaN,Widespread but patchy,NaN,NaN,Caribbean/GoM,NaN,NaN
100343.0,Belize,Half Moon Caye,NaN,17.150000,-87.566667,NaN,NaN,1995,20m,NaN,NaN,NaN,ReefBase,NaN,Widespread,NaN,NaN,Caribbean/GoM,NaN,NaN
100344.0,Belize,Turneffe,NaN,17.416667,-87.833333,NaN,NaN,1995,NaN,NaN,NaN,NaN,ReefBase,NaN,Widespread but patchy,NaN,NaN,Caribbean/GoM,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,Belize,NaN,West Snake Caye,16.190584,-88.569771,25.0,10.0,2016,NaN,22.39,NaN,In-situ,"Andrea Rivera Sosa from CINVESTAV, Melanie McF...",NaN,Depth and % bleached values are averaged acros...,"Andrea Rivera Sosa, CINVESTAV, Mexico",andrea.rivera@cinvestav.mx,Caribbean/GoM,22.39,22.39
NaN,Belize,NaN,Wilson,16.222947,-88.587921,27.0,10.0,2016,NaN,32.8,NaN,In-situ,"Andrea Rivera Sosa from CINVESTAV, Melanie McF...",NaN,Depth and % bleached values are averaged acros...,"Andrea Rivera Sosa, CINVESTAV, Mexico",andrea.rivera@cinvestav.mx,Caribbean/GoM,32.80,32.80
NaN,Belize,NaN,1117,17.383652,-87.518396,22.0,10.0,2017,NaN,60.38,NaN,In-situ,"Andrea Rivera Sosa from CINVESTAV, Melanie McF...",NaN,Depth and % bleached values are averaged acros...,"Andrea Rivera Sosa, CINVESTAV, Mexico",andrea.rivera@cinvestav.mx,Caribbean/GoM,60.38,60.38


In [49]:
def cumulativeLimit(x, lim):
    x[np.isnan(x)]=0
    total = 0.
    result = np.empty_like(x)
    for i, y in enumerate(x):
        total += y
        if total < lim:
            total = 0.
        result[i]=total
    return result

In [50]:
SummerAvgSST = pd.read_csv('SummerAvgSST.csv')

def extendSST(df):
    # This function takes the dataframe created from the MODIS values and creates the features for coefficient of variation (cv)
    # maximum temperature and the comparison to summertime averages.
	df['sst'] = df['sst'].interpolate()
	dfExt=pd.merge(df, SummerAvgSST, how = 'left', left_on = ['lon_Rnd','lat_Rnd'], right_on = ['Lon','Lat'])
	dfExt['sst_SumComp'] = dfExt['sst']-dfExt['sst_Summer']
	dfExt['sst_SumComp'] = dfExt['sst_SumComp'].fillna(0)
	dfExt['sst_sd'] = dfExt['sst'].rolling(14).std()
	dfExt['sst_mean'] = dfExt['sst'].rolling(14).mean()
	dfExt['sst_cv'] = dfExt['sst_sd'] * 100 / dfExt['sst_mean']

	dfExt['sst_streak'] = dfExt['sst_SumComp'].apply(np.floor)
	dfExt['sst_streak_min'] = dfExt['sst_streak'].rolling(7).min()
	dfExt.loc[dfExt['sst_streak_min'] < 1,'sst_streak_min'] = -1
	dfExt['sst_streak_min'] = dfExt['sst_streak_min'] / 7
	dfExt['DHW'] = cumulativeLimit(dfExt['sst_streak_min'].values,0)

	dfExt['sst_streak'] = dfExt['sst_SumComp'].apply(np.ceil)
	dfExt.loc[dfExt['sst_streak'] < 1,'sst_streak'] = -1
	dfExt['sst_streak'] = dfExt['sst_streak'] / 7
	dfExt['DHW'] = cumulativeLimit(dfExt['sst_streak_min'].values,0)

	return dfExt

In [59]:
def getModisBleachingFeatures(row):
	try:
		end_date = ee.Date(f"{int(row['year'])}-{int(row['month'])}-{int(row['day'])}")
		img_col = ee.ImageCollection("NASA/OCEANDATA/MODIS-Aqua/L3SMI")\
					.select(['chlor_a', 'nflh','poc', 'sst'])\
					.filterBounds(geometry=ee.Geometry.Point(row['long'], row['lat']))\
					.filterDate(end_date.advance(-1080, 'day'), end_date)\
					.getRegion(geometry=ee.Geometry.Point(row['long'], row['lat']), scale=1000)\
					.getInfo()

		df = pd.DataFrame(img_col[1:], columns=img_col[0])
		df.dropna(inplace=True)
		df['lat_Rnd'] = row['lat_Rnd']
		df['lon_Rnd'] = row['lat_Rnd']

		#Calculate statistics from 90 day history
		df = extendSST(df)
		df_90_Limit = df.tail(90)

		if df.shape[0] >= 1:
			row['chlor_max'] = np.max(df_90_Limit['chlor_a'])
			row['chlor_min'] = np.min(df_90_Limit['chlor_a'])
			row['chlor_avg'] = np.mean(df_90_Limit['chlor_a'])
			row['chlor_change'] = float(df_90_Limit['chlor_a'][-1:]) - float(df_90_Limit['chlor_a'][:1])
			row['nflh_max'] = np.max(df_90_Limit['nflh'])
			row['nflh_min'] = np.min(df_90_Limit['nflh'])
			row['nflh_avg'] = np.mean(df_90_Limit['nflh'])
			row['nflh_change'] = float(df_90_Limit['nflh'][-1:]) - float(df_90_Limit['nflh'][:1])
			row['poc_max'] = np.max(df_90_Limit['poc'])
			row['poc_min'] = np.min(df_90_Limit['poc'])
			row['poc_avg'] = np.mean(df_90_Limit['poc'])
			row['poc_change'] = float(df_90_Limit['poc'][-1:]) - float(df_90_Limit['poc'][:1])

			row['sst_day_of_study'] = df['sst'].tail(1).values[0]
			row['sst_max'] = df_90_Limit['sst'].max()
			row['sst_summer_max'] = df_90_Limit['sst_SumComp'].max()
			row['sst_cv_max'] = df_90_Limit['sst_cv'].max()
			row['sst_cv_cnt'] = df_90_Limit['sst_cv'].loc[df_90_Limit['sst_cv'] >= 1.9].count()
			row['sst_abv_summer'] = df_90_Limit['sst_SumComp'].loc[(df_90_Limit['sst_SumComp'] > 1)].count()
			row['sst_abv_summer_cumulative'] = df_90_Limit['sst_SumComp'].loc[(df_90_Limit['sst_SumComp']>1)].sum()
			row['sst_cv_cnt_SumComp'] = df_90_Limit['sst_cv'].loc[(df_90_Limit['sst_SumComp'] > 0) &
                                                                  (df_90_Limit['sst'] > df_90_Limit['sst_mean'])].count()
			row['sst_cv_max_SumComp'] = df_90_Limit['sst_cv'].loc[(df_90_Limit['sst_SumComp'] > 0) &
                                                                  (df_90_Limit['sst'] > df_90_Limit['sst_mean'])].max()
			row['sst_dhw'] = df['DHW'].max()
			row['sst_dhw_age'] = df.loc[df['DHW'] == df['DHW'].max()].index.values.astype(int)[0] - len(df)

		elif df.shape[0] == 0:
			row['chlor_max'] = np.nan
			row['chlor_min'] = np.nan
			row['chlor_avg'] = np.nan
			row['chlor_change'] = np.nan
			row['nflh_max'] = np.nan
			row['nflh_min'] = np.nan
			row['nflh_avg'] = np.nan
			row['nflh_change'] = np.nan
			row['poc_max'] = np.nan
			row['poc_min'] = np.nan
			row['poc_avg'] = np.nan
			row['poc_change'] = np.nan

			row['sst_day_of_study'] = np.nan
			row['sst_max'] = np.nan
			row['sst_summer_max'] = np.nan
			row['sst_cv_max'] = np.nan
			row['sst_cv_cnt'] = np.nan
			row['sst_abv_summer'] = np.nan
			row['sst_abv_summer_cumulative'] = np.nan
			row['sst_cv_cnt_SumComp'] = np.nan
			row['sst_cv_max_SumComp'] = np.nan
			row['sst_dhw'] = np.nan
			row['sst_dhw_age'] = np.nan

		return row

	except (ee.EEException, ValueError, KeyError, TypeError) as e:
		row['chlor_max'] = np.nan
		row['chlor_min'] = np.nan
		row['chlor_avg'] = np.nan
		row['chlor_change'] = np.nan
		row['nflh_max'] = np.nan
		row['nflh_min'] = np.nan
		row['nflh_avg'] = np.nan
		row['nflh_change'] = np.nan
		row['poc_max'] = np.nan
		row['poc_min'] = np.nan
		row['poc_avg'] = np.nan
		row['poc_change'] = np.nan

		row['sst_day_of_study'] = np.nan
		row['sst_max'] = np.nan
		row['sst_summer_max'] = np.nan
		row['sst_cv_max'] = np.nan
		row['sst_cv_cnt'] = np.nan
		row['sst_abv_summer'] = np.nan
		row['sst_abv_summer_cumulative'] = np.nan
		row['sst_cv_cnt_SumComp'] = np.nan
		row['sst_cv_max_SumComp'] = np.nan
		row['sst_dhw'] = np.nan
		row['sst_dhw_age'] = np.nan
		return row

In [60]:
df_GCBD.columns = df_GCBD.columns.str.lower()
df_bleaching = df_GCBD.swifter.apply(getModisBleachingFeatures, axis = 1)
df_bleaching

Pandas Apply:   0%|          | 0/569 [00:00<?, ?it/s]

,country,location,site_name,latitude,longitude,day,month,year,depth,percent_bleached,...,sst_max,sst_summer_max,sst_cv_max,sst_cv_cnt,sst_abv_summer,sst_abv_summer_cumulative,sst_cv_cnt_SumComp,sst_cv_max_SumComp,sst_dhw,sst_dhw_age
RECORD_ID,,,,,,,,,,,,,,,,,,,,,
100340.0,Belize,Ambergris Cay,NaN,18.033333,-87.883333,NaN,NaN,1995,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100341.0,Belize,NaN,NaN,17.316667,-88.000000,NaN,9.0,1995,42036,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100342.0,Belize,Carrie Bow Cay,NaN,16.800000,-88.083333,NaN,NaN,1995,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100343.0,Belize,Half Moon Caye,NaN,17.150000,-87.566667,NaN,NaN,1995,20m,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100344.0,Belize,Turneffe,NaN,17.416667,-87.833333,NaN,NaN,1995,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,Belize,NaN,West Snake Caye,16.190584,-88.569771,25.0,10.0,2016,NaN,22.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,Belize,NaN,Wilson,16.222947,-88.587921,27.0,10.0,2016,NaN,32.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,Belize,NaN,1117,17.383652,-87.518396,22.0,10.0,2017,NaN,60.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df_bleaching.to_pickle('gcbd_meshed_bleaching_features.pkl')

Index(['COUNTRY', 'LOCATION', 'SITE_NAME', 'LATITUDE', 'LONGITUDE', 'DAY',
       'MONTH', 'YEAR', 'DEPTH', 'PERCENT_BLEACHED', 'PERCENT_MORTALITY',
       'SURVEY_TYPE', 'SOURCE', 'CITATION', 'COMMENTS',
       'DATA_POINT_OF_CONTACT', 'POC_E-MAIL_ADDRESS', 'CORAL_REGIONS',
       'MIN_PERCENT_BLEACHED', 'MAX_PERCENT_BLEACHED'],
      dtype='object')